In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
#loading dataset 
data = pd.read_csv("dataset/Housing.csv")

In [ ]:
#print 5 values from the dataset
data.head(5)

In [ ]:
#checking null value in the dataset 
data.isnull().sum()

In [ ]:
#checking duplicate value in the dataset
data.duplicated()

In [ ]:
#print all unique value in each column of the dataset
for single_column in data.columns:
    print("unique value in {} are: {}".format(single_column,data[single_column].unique()))

In [ ]:
#print the name of columns which have values in the form of "yes" or "no"
columns_name = data.columns[data.isin(['yes', 'no']).any()]
print(columns_name)

In [ ]:
#converting "yes" or "no" values into "0" or "1"
for single_columns in columns_name:
    data[single_columns] = data[single_columns].replace({'yes': 1, 'no': 0},inplace=True)

In [ ]:
# convert string vlaue in "furnishingstatus" column in numerical "0", "1", "2"
data['furnishingstatus'] = data['furnishingstatus'].replace({'furnished': 2,'semi-furnished': 1,'unfurnished': 0,},inplace=True)

In [ ]:
data.head(5)

In [ ]:
#copying data to safe the real data
copied_data = data.copy()

In [ ]:
mean_value = copied_data.mean() #calculate mean of copied data
standard_deviation = copied_data.std() #claculate standard_deviation of copied data

In [ ]:
# Z_score formula = z = (data - mean value of data) / standard_deviation value of data
# claculate Z_score for each column
z_score = (copied_data - mean_value) / standard_deviation
print(z_score)

In [ ]:
# critical range lies between "-1.96" to "+1.96" 

#checking Z_score value lies between critical_range to check outlier
critical_range = ((z_score >= -1.96) & (z_score <= 1.96))

#convert checking values into percentage 
percentage = critical_range.mean() * 100
print("Percentage of values within [-1.96, 1.96] for each column:")

#print percentage value in ascending
print(percentage.sort_values(ascending=False).round(2))

### *`Z-Score Outlier Table Explained`*

1. **`basement` – 100% within range**
   → All values are within the normal Z-score range. No outliers detected.

2. **`airconditioning` – 100% within range**
   → Data is consistent. Every entry falls within expected boundaries.

3. **`prefarea` – 100% within range**
   → No unusual variation. Perfectly distributed data.

4. **`furnishingstatus` – 100% within range**
   → Uniform data distribution. No signs of outliers.

5. **`bathrooms` – 97.98% within range**
   → A small number of outliers (\~2%), possibly due to homes with more/fewer bathrooms than average.

6. **`parking` – 97.80% within range**
   → A few homes have more or fewer parking spaces than most, but still acceptable.

7. **`bedrooms` – 97.43% within range**
   → Slight deviation, indicating rare cases like unusually large or small homes.

8. **`area` – 95.60% within range**
   → Some outliers, likely due to significantly large property areas.

9. **`hotwaterheating` – 95.41% within range**
   → Few entries deviate from the norm, but not concerning.

10. **`price` – 95.05% within range**
    → A few high or low-priced houses are considered outliers but may still be valid.

11. **`stories` – 92.48% within range**
    → Moderate variation in the number of floors; some buildings have uncommon story counts.

12. **`mainroad` – 85.87% within range**
    → Slightly higher number of outliers; could reflect unusual access types.

13. **`guestroom` – 82.20% within range**
    → Highest outlier rate (\~18%). Might indicate that most homes either have or lack guest rooms, making the minority appear as outliers.

### *`Why Outlier Removal or Replace Is Not Needed`*

* Most columns have **95%+ values within the normal Z-score range**, showing data is largely well-behaved.
* Columns with slightly more outliers (e.g., `guestroom`, `mainroad`, `stories`) represent **real-world categorical or rare conditions**, not data errors.
* Outliers reflect **genuine property diversity** (e.g., large area, extra stories, premium price).
* Removing them could lead to **loss of important variation** and **reduced model performance**.

In [ ]:
sns.set(style="whitegrid")

In [ ]:
#ploting pearson correlation matrix heatmap
data_corr = data.corr(numeric_only=True)
plt.figure(figsize=(10, 8))
sns.heatmap(data_corr, annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("correlation heatmap")
plt.tight_layout()
plt.show()

### *`Correlation Heatmap`*

* `Area`, `bathrooms`, `bedrooms`, and `stories` have the strongest positive correlation with price.

* Features like `hotwaterheating` show a slight negative correlation, suggesting they don’t contribute much to pricing.

* `Furnishing status` has a low to moderate correlation (~0.30), but it still holds some influence.

* This heatmap helps us understand which features might be more useful in predictive modeling.

In [ ]:
#print column name which has vlaue more than 0.3 in correlation matrix excluding price 
target_corr = data_corr['price'].drop('price')
selected = target_corr[(target_corr) > 0.30].index
print("selected columns from correlation heatmap:")
print(selected.sort_values(ascending=False))

In [ ]:
# creating histogram Distribution of House-Price for checking outliers and skewness 
plt.figure(figsize=(10, 6))
sns.histplot(data["price"], kde=True, bins=30, color='skyblue')
plt.title("distribution of house prices")
plt.xlabel("price")
plt.ylabel("frequency")
plt.tight_layout()
plt.show()

### *`House Price Distribution`*

* The histogram shows that most houses are priced between `₹30 lakhs to ₹60 lakhs.` 

* The distribution is `right-skewed`, meaning there are a few very high-priced houses pushing the tail. 

* This implies that while affordable housing dominates, there are luxury properties too.


In [ ]:
#creating boxplot of categorical columns to do comparision with price 
categorical_columns = ['airconditioning', 'mainroad', 'basement', 'guestroom', 'hotwaterheating', 'furnishingstatus']
for columns in categorical_columns:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=columns, y='price', data=data)
    plt.title(f'Boxplot of Price vs {columns}')
    plt.xlabel(columns)
    plt.ylabel('Price')
    plt.show()

*The above boxplots show that houses with `air-conditioning`, access to the `main road`, and a `guestroom` tend to have higher median prices than those without. The presence of a `basement` shows only a slight price difference. `Hot water` heating appears to increase median price marginally, while the `furnishing-status` clearly impacts price — `fully-furnished houses` (label 2) have higher price ranges compared to `semi- or unfurnished` ones. so, these features contribute differently to house pricing, but `air-conditioning` and `furnishing-status` showing stronger influence.*


In [ ]:
#creating pairplot graph of high correlation feature with price show linearity and non-linearity ofn correlated fetures
selected_features = ['price', 'area', 'bathrooms', 'airconditioning', 'stories']
sns.pairplot(data[selected_features])
plt.suptitle('Pairplot of high Correlated Features with Price', y=1.02, fontsize=14)
plt.show()

*The above pairplot reveals that house price is positively correlated with `area`, indicating that larger properties tend to cost more. `Bathrooms` and number of `stories` also show a moderate upward trend with `price`. `Air-conditioning` appears as a binary factor but aligns more with `higher price` clusters. So the conclusion is the `area` shows the strongest `linear relationship` with price among the selected features.*


In [ ]:
#creating barplot in the comparision b/w mainroad and price 
plt.figure(figsize=(4, 4))
sns.barplot(x='mainroad', y='price', data=data)
plt.xticks([0, 1], ['not on mainroad', 'on mainroad'])
plt.title("price by mainroad")
plt.ylabel("price")
plt.xlabel("mainroad")
plt.tight_layout()
plt.show()

*The above bar-plot shows that houses located on the `main road` have significantly higher average `prices` than those that are not. This suggests that main road access positively influences property value, likely due to better connectivity and convenience.*


In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x='bedrooms', data=data, hue='bedrooms', palette='pastel', legend=False)
plt.title('Number of Bedrooms in Houses')
plt.xlabel('Bedrooms')
plt.ylabel('Count')
plt.show()

*The bar chart indicates that most houses have `3-bedrooms`, followed by those with `2 and 4 bedrooms`. Homes with `5 or more bedrooms` are rare. This suggests that `3-bedroom houses` are the most common and possibly the most preferred configuration in the dataset.*


In [ ]:
#scatter plot: area vs price
plt.figure(figsize=(8, 6))
sns.scatterplot(data=data, x='area', y='price', hue='furnishingstatus')
plt.title("scatter plot of area vs price")
plt.xlabel("area")
plt.ylabel("price")
plt.legend(title="furnished")
plt.tight_layout()
plt.show()

*The above scatter-plot shows a positive relationship between `area` and `price` — as `area increases, price generally rises`. Additionally, `fully-furnished homes` (category 2) tend to `cluster at higher prices`, It shows that furnishing level contributes to property value alongside area.*


In [ ]:
#countplots for categorical columns
categorical_features = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']

plt.figure(figsize=(18, 14))
for i, col in enumerate(categorical_features, 1):
    plt.subplot(3, 3, i)
    sns.countplot(x=col, data=data)
    plt.title(f"countplot of {col}")
    plt.xlabel(col)
    plt.ylabel("count")
    plt.tight_layout()

plt.suptitle("categorical feature distributions", fontsize=16)
plt.subplots_adjust(top=0.93)
plt.show()

*The above distribution plots show that most houses are located on the `main road`, have no `guest-room` or `basement`, and lack `hot water heating`. A significant number have no `air-conditioning` or `parking`. And majority of them are `semi-furnished`, with `unfurnished` and `fully-furnished` homes being less common.*
